In [1]:
from src.analysis import factorial_anova as fa

2025-03-15 14:48:30,116 - src.logging - INFO - Initializing...
2025-03-15 14:48:30,189 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.


In [2]:
results_path = '../math_reasoning_mock_results.jsonl'

results = fa.load_data(results_path)

results.head()

,template_id,n_varied_components,n_prompt_components_included,Role Assignment,Task Statement,Chain of Thought,Tree of Thoughts,Self-Verification,Example Solution,Problem Statement,prompt,question_id,response,correct
0,be6fbafc-fe85-4d71-8221-3be5a66da5dc,5,2,False,True,False,False,False,False,True,You are tasked with solving a math problem.\n\...,test/prealgebra/1353.json,idk,False
1,be6fbafc-fe85-4d71-8221-3be5a66da5dc,5,2,False,True,False,False,False,False,True,You are tasked with solving a math problem.\n\...,test/prealgebra/1646.json,idk,False
2,be6fbafc-fe85-4d71-8221-3be5a66da5dc,5,2,False,True,False,False,False,False,True,You are tasked with solving a math problem.\n\...,test/prealgebra/1787.json,idk,False
3,be6fbafc-fe85-4d71-8221-3be5a66da5dc,5,2,False,True,False,False,False,False,True,You are tasked with solving a math problem.\n\...,test/prealgebra/1423.json,idk,False
4,be6fbafc-fe85-4d71-8221-3be5a66da5dc,5,2,False,True,False,False,False,False,True,You are tasked with solving a math problem.\n\...,test/prealgebra/2086.json,idk,False


In [3]:
factor_columns = [
    'Role Assignment',
    'Chain of Thought',
    'Tree of Thoughts',
    'Self-Verification',
    'Example Solution',
]


metric_column = 'correct'

all_columns = factor_columns + [metric_column]

In [4]:
# Get sanitized column names
sanitized_metric = fa.sanitize_column_name(metric_column)
sanitized_factors = [
    fa.sanitize_column_name(factor) for factor in factor_columns
]
sanitized_columns = sanitized_factors + [sanitized_metric]

In [5]:
data = results[all_columns].copy().astype(int)
rename_dict = {i: j for i, j in zip(all_columns, sanitized_columns)}
data = data.rename(columns=rename_dict)
data.head()

,Role_Assignment,Chain_of_Thought,Tree_of_Thoughts,Self_Verification,Example_Solution,correct
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [6]:
formula = fa.get_formula(sanitized_metric, sanitized_factors)

print(formula)

correct ~ Role_Assignment * Chain_of_Thought * Tree_of_Thoughts * Self_Verification * Example_Solution


In [7]:
model = fa.get_fitted_model(data, formula)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                correct   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     17.47
Date:                Sat, 15 Mar 2025   Prob (F-statistic):           9.21e-80
Time:                        14:48:30   Log-Likelihood:                -333.43
No. Observations:                1600   AIC:                             730.9
Df Residuals:                    1568   BIC:                             903.0
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
========================================================================================================================================================
                                                                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                0.3000      0.043      7.046      0.000       0.216       0.384
Role_Assignment                                                                          0.3200      0.060      5.315      0.000       0.202       0.438
Chain_of_Thought                                                                         0.3200      0.060      5.315      0.000       0.202       0.438
Role_Assignment:Chain_of_Thought                                                        -0.0800      0.085     -0.939      0.348      -0.247       0.087
Tree_of_Thoughts                                                                         0.2600      0.060      4.318      0.000       0.142       0.378
Role_Assignment:Tree_of_Thoughts                                                        -0.0600      0.085     -0.705      0.481      -0.227       0.107
Chain_of_Thought:Tree_of_Thoughts                                                       -0.0800      0.085     -0.939      0.348      -0.247       0.087
Role_Assignment:Chain_of_Thought:Tree_of_Thoughts                                        0.0200      0.120      0.166      0.868      -0.216       0.256
Self_Verification                                                                        0.4000      0.060      6.643      0.000       0.282       0.518
Role_Assignment:Self_Verification                                                       -0.2200      0.085     -2.584      0.010      -0.387      -0.053
Chain_of_Thought:Self_Verification                                                      -0.1800      0.085     -2.114      0.035      -0.347      -0.013
Role_Assignment:Chain_of_Thought:Self_Verification                                       0.1400      0.120      1.163      0.245      -0.096       0.376
Tree_of_Thoughts:Self_Verification                                                      -0.2200      0.085     -2.584      0.010      -0.387      -0.053
Role_Assignment:Tree_of_Thoughts:Self_Verification                                       0.2200      0.120      1.827      0.068      -0.016       0.456
Chain_of_Thought:Tree_of_Thoughts:Self_Verification                                      0.2000      0.120      1.661      0.097      -0.036       0.436
Role_Assignment:Chain_of_Thought:Tree_of_Thoughts:Self_Verification                     -0.3400      0.170     -1.996      0.046      -0.674      -0.006
Example_Solution                                                                         0.2600      0.060      4.318      0.000       0.142       0.378
Role_Assignment:Example

In [8]:
anova_table = fa.run_factorial_anova_from_model(model)
anova_table

,sum_sq,df,F,PR(>F)
Role_Assignment,8.410000e+00,1.0,9.278694e+01,2.245183e-21
Chain_of_Thought,7.840000e+00,1.0,8.649817e+01,4.540329e-20
Role_Assignment:Chain_of_Thought,7.225000e-01,1.0,7.971292e+00,4.812554e-03
Tree_of_Thoughts,6.760000e+00,1.0,7.458261e+01,1.405980e-17
Role_Assignment:Tree_of_Thoughts,7.225000e-01,1.0,7.971292e+00,4.812554e-03
Chain_of_Thought:Tree_of_Thoughts,7.225000e-01,1.0,7.971292e+00,4.812554e-03
Role_Assignment:Chain_of_Thought:Tree_of_Thoughts,4.000000e-02,1.0,4.413172e-01,5.065851e-01
Self_Verification,7.840000e+00,1.0,8.649817e+01,4.540329e-20
Role_Assignment:Self_Verification,1.322500e+00,1.0,1.459105e+01,1.387565e-04
Chain_of_Thought:Self_Verification,7.225000e-01,1.0,7.971292e+00,4.812554e-03
